In [ ]:
'''
user
mat
tag
age (~20, 21-30, 31-40, 41-50, 51-60, 60~)
eng_lv (1,2,3,...,12)
'''

In [ ]:
'''
1. load data
2. entity_name2idx
'''

In [107]:
# load base data
import pickle
path = 'organic_class_kg1_2022_1_3.pkl'
with open(path, "rb") as f:
    basic_info = pickle.load(f)
basic_info.keys()

dict_keys(['class_dat', 'mat2dcgs_tag', 'mat2eng_level', 'mat2title_sm', 'user2dcgs_tag', 'user2age', 'user2eng_level'])

In [108]:
class_dat = basic_info['class_dat']
class_dat.head(3)

,client_sn,attend_level,attend_datetime,MaterialID,M_Point
30824,382034,12,2022-01-01 00:30:00,136057,10.0
36830,241953,12,2022-01-01 02:30:00,120129,10.0
31458,2984802,12,2022-01-01 05:30:00,106151,10.0


In [109]:
max(class_dat['attend_datetime'])

Timestamp('2022-03-25 10:30:00')

In [110]:
train_ed_date = '2022-03-10 00:30:00'
val_ed_date = '2022-03-15 00:30:00'
test_ed_date = '2022-03-26 00:30:00'

In [111]:
train_class_dat = class_dat[class_dat['attend_datetime'] <= train_ed_date]
val_class_dat = class_dat[(class_dat['attend_datetime'] <= val_ed_date) & 
                            (class_dat['attend_datetime'] > train_ed_date)]
test_class_dat = class_dat[(class_dat['attend_datetime'] <= test_ed_date) & 
                           (class_dat['attend_datetime'] > val_ed_date)]

In [112]:
train_class_dat.shape

(65695, 5)

In [113]:
val_class_dat.shape

(5772, 5)

In [114]:
test_class_dat.shape

(12134, 5)

In [115]:
train_triple_data = [[0,0,0]]
val_triple_data = []
test_triple_data = []
entity_idx = 1
entity_name2idx = {'padding':0}
relation_name2idx = {'user-prefer_mat' : 0, 
                     'user-tag' : 1,
                     'mat-tag' : 2,
                     'mat-englv' : 3,
                     'user-age' : 4}

In [116]:
# collect max score for user
user2max_score = dict()
dat = class_dat.groupby(by='client_sn').max()
dat['client_sn'] = dat.index
for element in dat.to_dict('reocrds'):
    client_sn = element['client_sn']
    M_Point = element['M_Point']
    user2max_score[client_sn] = M_Point

/home/hsucheng/.local/lib/python3.6/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


In [117]:
def build_interaction_hrt(t_class_dat, user2max_score, entity_name2idx, triple_data, entity_idx):
    for element in list(t_class_dat.to_dict('records')):
        client_sn = element['client_sn']
        MaterialID = element['MaterialID']
        M_Point = element['M_Point'] 
        if M_Point >= user2max_score[client_sn]:
            entity_name = str(client_sn)+'@'+'client_sn'
            if entity_name not in entity_name2idx:
                entity_name2idx[entity_name] = entity_idx
                entity_idx +=1
            entity_name = str(MaterialID)+'@'+'MaterialID'
            if entity_name not in entity_name2idx:
                entity_name2idx[entity_name] = entity_idx
                entity_idx +=1   
            h_idx = entity_name2idx[str(client_sn)+'@'+'client_sn']
            t_idx = entity_name2idx[str(MaterialID)+'@'+'MaterialID']
            r_idx = relation_name2idx['user-prefer_mat']
            triple_data.append([h_idx, t_idx, r_idx])
    return entity_name2idx, triple_data, entity_idx

In [118]:
# train
entity_name2idx, train_triple_data, entity_idx = \
    build_interaction_hrt(train_class_dat, user2max_score, entity_name2idx, train_triple_data, entity_idx)
# val
entity_name2idx, val_triple_data, entity_idx = \
    build_interaction_hrt(val_class_dat, user2max_score, entity_name2idx, val_triple_data, entity_idx)
# test
entity_name2idx, test_triple_data, entity_idx = \
    build_interaction_hrt(test_class_dat, user2max_score, entity_name2idx, test_triple_data, entity_idx)

In [119]:
user2dcgs_tag = basic_info['user2dcgs_tag']
mat2dcgs_tag = basic_info['mat2dcgs_tag']
mat2eng_level = basic_info['mat2eng_level']
user2age = basic_info['user2age']

In [120]:
def build_user_data_hrt(t_class_dat, user2dcgs_tag, entity_name2idx, entity_idx, triple_data, user2age):
    for user in list(set(t_class_dat['client_sn'])):
        # tag
        if user in user2dcgs_tag:
            tagList = list(user2dcgs_tag[user])
        else:
            tagList = []
        for tag in tagList:
            entity_name = str(tag)+'@'+'dcgs_tag'
            if entity_name not in entity_name2idx:
                entity_name2idx[entity_name] = entity_idx
                entity_idx +=1
            h_idx = entity_name2idx[str(user)+'@'+'client_sn']
            t_idx = entity_name2idx[str(tag)+'@'+'dcgs_tag']
            r_idx = relation_name2idx['user-tag']
            triple_data.append([h_idx, t_idx, r_idx])
        # age
        if user in user2age:
            age = user2age[user]
        else:
            age = 0
        if age <= 20:
            ageID = 0
        elif 21 <= age <= 30:
            ageID = 1
        elif 31 <= age <= 40:
            ageID = 2
        elif 41 <= age <= 50:
            ageID = 3
        elif 51 <= age <= 60:
            ageID = 4
        else:
            ageID = 5
        entity_name = str(ageID)+'@'+'user_age'
        if entity_name not in entity_name2idx:
            entity_name2idx[entity_name] = entity_idx
            entity_idx +=1
        h_idx = entity_name2idx[str(user)+'@'+'client_sn']
        t_idx = entity_name2idx[str(ageID)+'@'+'user_age']
        r_idx = relation_name2idx['user-age']
        triple_data.append([h_idx, t_idx, r_idx])
  
    return entity_name2idx, triple_data, entity_idx

In [121]:
# user tag
entity_name2idx, train_triple_data, entity_idx = \
    build_user_data_hrt(train_class_dat, user2dcgs_tag, entity_name2idx, entity_idx, train_triple_data, user2age)
entity_name2idx, val_triple_data, entity_idx = \
    build_user_data_hrt(val_class_dat, user2dcgs_tag, entity_name2idx, entity_idx, val_triple_data ,user2age)
entity_name2idx, test_triple_data, entity_idx = \
    build_user_data_hrt(test_class_dat, user2dcgs_tag, entity_name2idx, entity_idx, test_triple_data, user2age)

In [122]:
def build_mat_data_hrt(t_class_dat, mat2dcgs_tag, entity_name2idx, entity_idx, triple_data, mat2eng_level):
    for mat in list(set(t_class_dat['MaterialID'])):
        # tag
        if mat in mat2dcgs_tag and str(mat)+'@'+'MaterialID' in entity_name2idx:
            tagList = list(mat2dcgs_tag[mat])
        else:
            tagList = []
        for tag in tagList:
            entity_name = str(tag)+'@'+'dcgs_tag'
            if entity_name not in entity_name2idx:
                entity_name2idx[entity_name] = entity_idx
                entity_idx +=1
            h_idx = entity_name2idx[str(mat)+'@'+'MaterialID']
            t_idx = entity_name2idx[str(tag)+'@'+'dcgs_tag']
            r_idx = relation_name2idx['user-tag']
            triple_data.append([h_idx, t_idx, r_idx])
        # lv
        if mat in mat2eng_level and str(mat)+'@'+'MaterialID' in entity_name2idx:
            try:
                lvList = list(mat2eng_level[mat])
            except:
                lvList = []
        else:
            lvList = []
        for lv in lvList:
            entity_name = str(lv)+'@'+'eng_lv'
            if entity_name not in entity_name2idx:
                entity_name2idx[entity_name] = entity_idx
                entity_idx +=1
            h_idx = entity_name2idx[str(mat)+'@'+'MaterialID']
            t_idx = entity_name2idx[str(lv)+'@'+'eng_lv']
            r_idx = relation_name2idx['mat-englv']
            triple_data.append([h_idx, t_idx, r_idx])
    return entity_name2idx, triple_data, entity_idx

In [123]:
# mat tag
entity_name2idx, train_triple_data, entity_idx = \
    build_mat_data_hrt(train_class_dat, mat2dcgs_tag, entity_name2idx, entity_idx, train_triple_data, mat2eng_level)
entity_name2idx, val_triple_data, entity_idx = \
    build_mat_data_hrt(val_class_dat, mat2dcgs_tag, entity_name2idx, entity_idx, val_triple_data, mat2eng_level)
entity_name2idx, test_triple_data, entity_idx = \
    build_mat_data_hrt(test_class_dat, mat2dcgs_tag, entity_name2idx, entity_idx, test_triple_data, mat2eng_level)

0

In [124]:
len(entity_name2idx)

13700

In [125]:
len(train_triple_data)

114370

In [126]:
len(val_triple_data)

31580

In [127]:
len(test_triple_data)

46785

In [128]:
len(relation_name2idx)

5

In [ ]:
'''
Upstream
1. OUTPUT : user_emb, item_emb [V]
2. EVAL : user instance can be closed with matched tag instance [V]
3. EVAL : user instance can be closed with favor item instance [V ; metric should be same with downstream]
4. EVAL : item instance can be closed with matched tag instance [V]
Downstram
1. OUTPUT : pred_item LIST [V]
2. EVAL : basic rank performance [V]
3. EVAL : seq_item can be more semantic [?]
'''

In [131]:
import pickle

data = {'entity_name2idx' : entity_name2idx,
        'relation_name2idx' : relation_name2idx,
        'train_triple_data' : train_triple_data,
        'val_triple_data' : val_triple_data,
        'test_triple_data' : test_triple_data}

with open('KG_class_data_padding.pkl', 'wb') as f:
    pickle.dump(data, f)

In [ ]:
'''
user
mat
tag
age (~20, 21-30, 31-40, 41-50, 51-60, 60~)
eng_lv (1,2,3,...,12)
'''
'''
user->mat : user-prefer_mat
user->tag : user-tag
mat->tag : mat-tag
mat-lv : mat-englv
user-age : user-age
'''